https://nvidia-ai-iot.github.io/jetson-min-disk/minimal-l4t.html
# Option: Starting with Minimal L4T

If you are using an x86-64 Linux host PC to flash your Jetson, you can create a minimal configuration RootFS and flash that image onto Jetson.

In the diagram shown earlier (“Varying degree of minimized configurations”), this corresponds to using L4T Flash Tool to flash Minimal L4T BSP to directly get to Configuration [b].

## How to Flash Minimal L4T BSP

For setting up the flashing tool on a host PC, you can either download individual tar files from the NVIDIA website, or use NVIDIA SDK Manager to download and set up the flashing tool.

### Using Tar Files (CREATE LINUX FOR TEGRA)

If you opt to take the first route, go to the L4T archive page, click on the L4T version that you want to use, and jump to the individual L4T version page.

Download both L4T Driver Package (BSP) and Sample Root Filesystem on your host PC.

**Example for downloading and setting up L4T Flash Tool on Host PC:**
```bash
sudo apt-get install qemu-user-static
```
```bash
wget https://developer.nvidia.com/embedded/l4t/r32_release_v7.1/t186/jetson_linux_r32.7.1_aarch64.tbz2
wget https://developer.nvidia.com/embedded/l4t/r32_release_v7.1/t186/tegra_linux_sample-root-filesystem_r32.7.1_aarch64.tbz2
L4T_RELEASE_PACKAGE=Jetson-210_Linux_R32.7.4_aarch64.tbz2
SAMPLE_FS_PACKAGE=Tegra_Linux_Sample-Root-Filesystem_R32.7.4_aarch64.tbz2
tar xf ${L4T_RELEASE_PACKAGE}
cd Linux_for_Tegra/rootfs/
sudo tar xpf ../../${SAMPLE_FS_PACKAGE}
cd ..
sudo ./apply_binaries.sh
```

```text
➜  RootFS L4T_RELEASE_PACKAGE=Jetson-210_Linux_R32.7.4_aarch64.tbz2
SAMPLE_FS_PACKAGE=Tegra_Linux_Sample-Root-Filesystem_R32.7.4_aarch64.tbz2
tar xf ${L4T_RELEASE_PACKAGE}
cd Linux_for_Tegra/rootfs/
sudo tar xpf ../../${SAMPLE_FS_PACKAGE}
cd ..
sudo ./apply_binaries.sh
Using rootfs directory of: /home/rsemihkoca/Downloads/RootFS/Linux_for_Tegra/rootfs
Installing extlinux.conf into /boot/extlinux in target rootfs
/home/rsemihkoca/Downloads/RootFS/Linux_for_Tegra/nv_tegra/nv-apply-debs.sh
Root file system directory is /home/rsemihkoca/Downloads/RootFS/Linux_for_Tegra/rootfs
Copying public debian packages to rootfs
Start L4T BSP package installation
QEMU binary is not available, looking for QEMU from host system
Found /usr/bin/qemu-aarch64-static
Installing QEMU binary in rootfs
/home/rsemihkoca/Downloads/RootFS/Linux_for_Tegra/rootfs /home/rsemihkoca/Downloads/RootFS/Linux_for_Tegra
Installing BSP Debian packages in /home/rsemihkoca/Downloads/RootFS/Linux_for_Tegra/rootfs
Selecting previously unselected package nvidia-l4t-core.
(Reading database ... 142273 files and directories currently installed.)
Preparing to unpack .../nvidia-l4t-core_32.7.4-20230608212426_arm64.deb ...
Pre-installing... skip compatibility checking.
Unpacking nvidia-l4t-core (32.7.4-20230608212426) ...
Setting up nvidia-l4t-core (32.7.4-20230608212426) ...
Selecting previously unselected package jetson-gpio-common.
(Reading database ... 142319 files and directories currently installed.)
Preparing to unpack .../jetson-gpio-common_2.0.17_arm64.deb ...
Unpacking jetson-gpio-common (2.0.17) ...
...
Setting up nvidia-l4t-initrd (32.7.4-20230608212426) ...
Pre-installing initrd package, skip flashing
Setting up nvidia-l4t-jetson-io (32.7.4-20230608212426) ...
Setting up nvidia-l4t-multimedia (32.7.4-20230608212426) ...
Setting up nvidia-l4t-camera (32.7.4-20230608212426) ...
Setting up nvidia-l4t-gstreamer (32.7.4-20230608212426) ...
Processing triggers for nvidia-l4t-kernel (4.9.337-tegra-32.7.4-20230608212426) ...
Processing triggers for libc-bin (2.27-3ubuntu1.6) ...
/home/rsemihkoca/Downloads/RootFS/Linux_for_Tegra
Removing QEMU binary from rootfs
Removing stashed Debian packages from rootfs
L4T BSP package installation completed!
Rename ubuntu.desktop --> ux-ubuntu.desktop
Disabling NetworkManager-wait-online.service
Disable the ondemand service by changing the runlevels to 'K'
Success!
```


## Create Custom RootFS Image Using `nv_build_samplefs.sh`

Regardless of whether you set up the flashing tool (`Linux_for_Tegra` directory) using tar files or using SDK Manager, L4T Flash Tool comes with a pre-built sample root file system.

NVIDIA also provides a tool to generate a root file system image, `nv_build_samplefs.sh`.

### Usage of `nv_build_samplefs.sh`

`nv_build_samplefs.sh` usage: r32.x vs r34.x

```bash
cd Linux_for_Tegra/tools/samplefs/
ls
nv_build_samplefs.sh  nvubuntu-bionic-aarch64-packages  nvubuntu-bionic-aarch64-samplefs  nvubuntu_samplefs.sh
sudo ./nv_build_samplefs.sh 
Usage:
nv_build_samplefs.sh --abi <ABI> --distro <distro> --version <version> [--verbose]
        <ABI>           - The ABI of Linux distro. Such as 'aarch64'
        <distro>        - The Linux distro. Such as 'ubuntu'
        <version>       - The version of Linux distro. Such as 'bionic' for Ubuntu.
Example:
nv_build_samplefs.sh --abi aarch64 --distro ubuntu --version bionic

nv_build_samplefs.sh will download the base image for the given Linux distro, install necessary
packages, and generate samplefs tarball, so an internet connection is required.

Generated samplefs tarball will be named 'sample_fs.tbz2' and put under the path
executes this script.
```

### Customize RootFS Composition/Flavor

As seen in the above usage, `nv_build_samplefs.sh` has a slight difference:

- **r32.x**: You need to edit `nvubuntu-bionic-aarch64-packages`.
- **r34.x**: You can use a custom package list with the `--flavor` option.

L4T (Jetson Linux) 32.7.2 (for JetPack 4.6.2) or earlier has a version of `nv_build_samplefs.sh` that takes the text file `nvubuntu-bionic-aarch64-packages` that lists out the public Debian packages to be installed in the rootfs.

You can edit this package file to build your own minimal rootfs image. For example, here is an example list of packages that would generate a minimal configuration rootfs image without Ubuntu desktop GUI for Rel-32.7.


L4T (Jetson Linux) 34.1 (for JetPack 5.0 DP) is based on Ubuntu 20.04, and we have a new version of `nv_build_samplefs.sh` that takes the flavor option.

You will find text files `nvubuntu-focal-desktop-aarch64-packages` and `nvubuntu-focal-minimal-aarch64-packages` in the samplefs directory. You can use `--flavor {CUSTOM}` to specify `nvubuntu-focal-{CUSTOM}-aarch64-packages` file.

**Example commands to create custom composition RootFS image:**

```
change nvubuntu-bionic-aarch64-packages
My packages are in https://gist.github.com/rsemihkoca/44030997ae1a89989d38d1aebe0fb383

```

### Create sample_fs.tbz2 

docker container ve host arasındaki volume içinde yaptığın değişiklikler host makinesine de yansır.
// update packages here from above gist
'''bash
sudo docker run --privileged -it --rm -v ./Linux_for_Tegra:/l4t ubuntu:18.04
sudo docker run --privileged -it --rm -v ./Linux_for_Tegra:/l4t -v  /run/dbus/system_bus_socket:/run/dbus/system_bus_socket:ro ubuntu:18.04
// inside docker:
apt-get update
apt-get install -y qemu-user-static wget sudo systemd nano
cd /l4t/tools/samplefs

```

```bash
cd Linux_for_Tegra/tools/samplefs/
mv nvubuntu-bionic-aarch64-packages nvubuntu-bionic-aarch64-packages.orig
mv ~/Downloads/nvubuntu-bionic-minimal-aarch64-packages ./nvubuntu-bionic-aarch64-packages
sudo ./nv_build_samplefs.sh --abi aarch64 --distro ubuntu --version bionic --verbose | tee nv_build_samplefs.log


SAMPLE_FS_PACKAGE=$(pwd)/sample_fs.tbz2
cd ../../../
cd Linux_for_Tegra/rootfs/
sudo rm -rf ./*/
sudo tar xpf ${SAMPLE_FS_PACKAGE}
cd ..
sudo ./apply_binaries.sh
cd ./tools/
sudo ./l4t_create_default_user.sh -u rsemihkoca -p 8520 -n jetson_nano -a --accept-license

sudo ./l4t_create_default_user.sh -u {USERNAME} -p {PASSWORD} -n {HOSTNAME} ‑‑accept-license --autologin
cd ..
```

## After sample_fs.tbz2  you can look in sh_into_tarball.ipynb for sh into this rootfs befor turn it into image


## Create Boot image

https://docs.nvidia.com/jetson/archives/l4t-archived/l4t-3271/index.html#page/Tegra%20Linux%20Driver%20Package%20Development%20Guide/flashing.html#wwpID0E0BL0HA

 sudo ROOTFS_DIR=/home/rsemihkoca/Downloads/RootFS/Linux_for_Tegra/rootfs ./jetson-disk-image-creator.sh -o sd-blob.img -b jetson-nano -r 300


## Flash and Boot

**Example command to flash Jetson Xavier NX production module from a Host PC:**


```bash
sudo ./flash.sh jetson-xavier-nx-devkit-emmc internal
```

See the developer guide (r32.7, r34.1) to flash your Jetson.

You can log into the Jetson using the username and password you used when running `l4t_create_default_user.sh` script.

**Attention:** Note you may need to manually bring "up" `eth0` if you removed the network-manager package in order to enable Ethernet network access.

## Results

You can check the disk usage and the install package size using some of the commands introduced in the [Analyze Disk Usage](https://developer.nvidia.com/embedded/learn/get-started-jetson-nano-devkit#analyze-disk-usage) page.

**Example output of `df` and `dpkg` commands on Jetson:**

```bash
Filesystem     1K-blocks    Used Available Use% Mounted on
/dev/mmcblk0p1  14384136 1756760  11876992  13% /
Filesystem      Size  Used Avail Use% Mounted on
/dev/mmcblk0p1   14G  1.7G   12G  13% /
```

```bash
613
     624 KiB    adduser                         add and remove users and groups
    5399 KiB    adwaita-icon-theme              default icon theme of GNOME (small subset)
    2240 KiB    alsa-utils                      Utilities for configuring and using ALSA
     792 KiB    apport                          automatically generate crash reports for debugging
      75 KiB    apport-symptoms                 symptom scripts for apport
    3891 KiB    apt                             commandline package manager
     276 KiB    at-spi2-core                    Assistive Technology Service Provider Interface (dbus core)
     387 KiB    base-files                      Debian base system miscellaneous files
     228 KiB    base-passwd                     Debian base system master password and group files
    1528 KiB    bash                            GNU Bourne Again SHell
     214 KiB    bc                              GNU bc arbitrary precision calculator language
      99 KiB    bridge-utils                    Utilities for configuring the Linux Ethernet bridge
     248 KiB    bsdutils                        basic utilities from 4.4BSD-Lite
     174 KiB    bzip2                           high-quality block-sorting file compressor - utilities
     374 KiB    ca-certificates                 Common CA certificates
     500 KiB    can-utils                       SocketCAN userspace utilities and tools
     446 KiB    console-setup                   console font and keymap setup program
    1229 KiB    console-setup-linux             Linux specific part of console-setup
    6052 KiB    coreutils                       GNU core utilities
      64 KiB    cpp                             GNU C preprocessor (cpp)
   17973 KiB    cpp-7                           GNU C preprocessor
     269 KiB    crda                            wireless Central Regulatory Domain Agent
     247 KiB    cron                           


# Option: Starting with Minimal L4T

If you are using an x86-64 Linux host PC to flash your Jetson, you can create a minimal configuration RootFS and flash that image onto Jetson.

In the diagram shown earlier (“Varying degree of minimized configurations”), this corresponds to using L4T Flash Tool to flash Minimal L4T BSP to directly get to Configuration [b].

## How to Flash Minimal L4T BSP

For setting up the flashing tool on a host PC, you can either download individual tar files from the NVIDIA website, or use NVIDIA SDK Manager to download and set up the flashing tool.

### Using Tar Files

If you opt to take the first route, go to the L4T archive page, click on the L4T version that you want to use, and jump to the individual L4T version page.

Download both L4T Driver Package (BSP) and Sample Root Filesystem on your host PC.

**Example for downloading and setting up L4T Flash Tool on Host PC:**

```bash
wget https://developer.nvidia.com/embedded/l4t/r32_release_v7.1/t186/jetson_linux_r32.7.1_aarch64.tbz2
wget https://developer.nvidia.com/embedded/l4t/r32_release_v7.1/t186/tegra_linux_sample-root-filesystem_r32.7.1_aarch64.tbz2
L4T_RELEASE_PACKAGE=jetson_linux_r32.7.1_aarch64.tbz2
SAMPLE_FS_PACKAGE=tegra_linux_sample-root-filesystem_r32.7.1_aarch64.tbz2
tar xf ${L4T_RELEASE_PACKAGE}
cd Linux_for_Tegra/rootfs/
sudo tar xpf ../../${SAMPLE_FS_PACKAGE}
cd ..
sudo ./apply_binaries.sh
```

### Using SDK Manager

You can also use NVIDIA SDK Manager to download the flashing tool. Follow the "Download and Run SDK Manager" page and "Install Jetson Software with SDK Manager" page of SDK Manager Documentation, go through the flashing process until "SDK Manager is about to flash your Jetson ~" dialog comes up. Click the "Skip" button and exit from SDK Manager.

You should find the `Linux_for_Tegra` directory set up in the following directory.

**Example of using SDK Manager to find L4T Flashing Tool for Jetson AGX Xavier:**

```bash
cd ~/nvidia/nvidia_sdk/JetPack_4.6.1_Linux_JETSON_AGX_XAVIER_TARGETS
cd Linux_for_Tegra
```

## Create Custom RootFS Image Using `nv_build_samplefs.sh`

Regardless of whether you set up the flashing tool (`Linux_for_Tegra` directory) using tar files or using SDK Manager, L4T Flash Tool comes with a pre-built sample root file system.

NVIDIA also provides a tool to generate a root file system image, `nv_build_samplefs.sh`.

### Usage of `nv_build_samplefs.sh`

`nv_build_samplefs.sh` usage: r32.x vs r34.x

```bash
$ cd Linux_for_Tegra/tools/samplefs/
$ ls
nv_build_samplefs.sh  nvubuntu-bionic-aarch64-packages  nvubuntu-bionic-aarch64-samplefs  nvubuntu_samplefs.sh
$ sudo ./nv_build_samplefs.sh 
Usage:
nv_build_samplefs.sh --abi <ABI> --distro <distro> --version <version> [--verbose]
        <ABI>           - The ABI of Linux distro. Such as 'aarch64'
        <distro>        - The Linux distro. Such as 'ubuntu'
        <version>       - The version of Linux distro. Such as 'bionic' for Ubuntu.
Example:
nv_build_samplefs.sh --abi aarch64 --distro ubuntu --version bionic

nv_build_samplefs.sh will download the base image for the given Linux distro, install necessary
packages, and generate samplefs tarball, so an internet connection is required.

Generated samplefs tarball will be named 'sample_fs.tbz2' and put under the path
executes this script.
```

### Customize RootFS Composition/Flavor

As seen in the above usage, `nv_build_samplefs.sh` has a slight difference:

- **r32.x**: You need to edit `nvubuntu-bionic-aarch64-packages`.
- **r34.x**: You can use a custom package list with the `--flavor` option.

L4T (Jetson Linux) 32.7.2 (for JetPack 4.6.2) or earlier has a version of `nv_build_samplefs.sh` that takes the text file `nvubuntu-bionic-aarch64-packages` that lists out the public Debian packages to be installed in the rootfs.

You can edit this package file to build your own minimal rootfs image. For example, here is an example list of packages that would generate a minimal configuration rootfs image without Ubuntu desktop GUI for Rel-32.7.

L4T (Jetson Linux) 34.1 (for JetPack 5.0 DP) is based on Ubuntu 20.04, and we have a new version of `nv_build_samplefs.sh` that takes the flavor option.

You will find text files `nvubuntu-focal-desktop-aarch64-packages` and `nvubuntu-focal-minimal-aarch64-packages` in the samplefs directory. You can use `--flavor {CUSTOM}` to specify `nvubuntu-focal-{CUSTOM}-aarch64-packages` file.

**Example commands to create custom composition RootFS image:**

```bash
cd Linux_for_Tegra/tools/samplefs/
mv nvubuntu-bionic-aarch64-packages nvubuntu-bionic-aarch64-packages.orig
mv ~/Downloads/nvubuntu-bionic-minimal-aarch64-packages ./nvubuntu-bionic-aarch64-packages
sudo ./nv_build_samplefs.sh --abi aarch64 --distro ubuntu --version bionic | tee nv_build_samplefs.log
SAMPLE_FS_PACKAGE=$(pwd)/sample_fs.tbz2
cd ../../../
cd Linux_for_Tegra/rootfs/
sudo rm -rf ./*/
sudo tar xpf ${SAMPLE_FS_PACKAGE}
cd ..
sudo ./apply_binaries.sh
cd ./tools/
sudo ./l4t_create_default_user.sh -u {USERNAME} -p {PASSWORD} -n {HOSTNAME} ‑‑accept-license --autologin
cd ..
```

## Flash and Boot

**Example command to flash Jetson Xavier NX production module from a Host PC:**

```bash
sudo ./flash.sh jetson-xavier-nx-devkit-emmc internal
```

See the developer guide (r32.7, r34.1) to flash your Jetson.

You can log into the Jetson using the username and password you used when running `l4t_create_default_user.sh` script.

**Attention:** Note you may need to manually bring "up" `eth0` if you removed the network-manager package in order to enable Ethernet network access.

## Results

You can check the disk usage and the install package size using some of the commands introduced in the [Analyze Disk Usage](https://developer.nvidia.com/embedded/learn/get-started-jetson-nano-devkit#analyze-disk-usage) page.

**Example output of `df` and `dpkg` commands on Jetson:**

```bash
Filesystem     1K-blocks    Used Available Use% Mounted on
/dev/mmcblk0p1  14384136 1756760  11876992  13% /
Filesystem      Size  Used Avail Use% Mounted on
/dev/mmcblk0p1   14G  1.7G   12G  13% /
```

```bash
613
     624 KiB    adduser                         add and remove users and groups
```